Use continuous geographic, hydrological, climate, and meteorological datasets as input to train neural network(NN) and random forest(RF) models, to determine the major environmentl drivers of algae growth in lakes.
Use Google Cloud Platform (GCP) to extract relative data from its Earth Engine Dataset and run further NN models

In [1]:
##connect to google earth engine
import ee
service_account = '429406912917-compute@developer.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'credentials.json')
ee.Initialize(credentials)

In [86]:
##prepare a mapping tool
import folium
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [88]:
### extract climate data from ECMWF ERA 5
##details of dataset: https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY#description
## https://cds.climate.copernicus.eu/cdsapp#!/software/app-era5-explorer?tab=overview
sensor='ECMWF/ERA5/DAILY'
start_date,end_date='2020-06-01','2020-07-01'
aoi =ee.Geometry.Rectangle(-95.4, 48.88,-93.75, 49.77)
era5 = ee.ImageCollection(sensor) \
       .filterBounds(aoi)  \
       .filterDate(ee.Date(start_date), ee.Date(end_date))
era5_2mt = era5.select('mean_2m_air_temperature')
print('Number of images', era5_2mt.size().getInfo())

Number of images 30


In [122]:
##preview the average imagery
avgT=era5_2mt.mosaic().add(-273.15)  #kelvin to Celsius  c=k-273.15   #.clip(aoi)
##result = ee.Image.expression('(slope * aspect) / 255', {'slope': slope,'aspect': aspect})
print("Global temperature Jun 2020 monthly avg (C)")

## #show interactive map
## vis_params = {
##   'min': -30,
##   'max': 40,
##   'palette': ['blue', 'green', 'red']
## }
## center = aoi.centroid()
## map = folium.Map(location=[center.coordinates().get(1).getInfo(),center.coordinates().get(0).getInfo()], zoom_start=5)
## #map = folium.Map(location=[48.88,-95.4], zoom_start=10)
## # Add the image layer to the map and display it.
## map.add_ee_layer(avgT, vis_params, 'Avg Temp (c)')
## map.add_child(folium.LayerControl())
## display(map)

##display a raster imagery
from IPython.display import Image
Image(url =avgT.updateMask(avgT.gt(-10)).getThumbURL({'min':0, 'max': 40, 'dimensions': 512,
                'palette': ['blue', 'green', 'red']}))

Global temperature Jun 2020 monthly avg (C)


In [6]:
## export to a test imagery to Google Cloud Platform (GCP) buckets 
img=era5_2mt.first()
export_config = {
    'image': img,
    'region': aoi,
    'crs': 'EPSG:4326',
    'scale': 100000,    
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': 'ERA5_2mt',
    'bucket':   'AlgaeDrive',  
    'maxPixels':1e10,
    'description': 'ERA5 climate data download: ER5 mean_2m_air_temperature'
}
task=ee.batch.Export.image.toCloudStorage(**export_config)
#task=ee.batch.Export.image.toDrive(**export_config)
task.start()
import time
time.sleep(10)
print(task.status())

{'state': 'READY', 'description': 'ERA5 climate data download: ER5 mean_2m_air_temperature', 'creation_timestamp_ms': 1680028182464, 'update_timestamp_ms': 1680028182464, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UQFQZPCG5XE5SKJFP2QS4TKZ', 'name': 'projects/earthengine-legacy/operations/UQFQZPCG5XE5SKJFP2QS4TKZ'}


In [ ]:
##iterate all the image to export to GCP bucket
crs_epsg=4326
img_list = era5_2mt.toList(era5_2mt.size())
nImages = img_list.size().getInfo()
for i in range(nImages):
	img=ee.Image(img_list.get(i)).clip(aoi)
	product_id=img.getInfo()['id'].replace('/','_')
	export_config = {
		'image': img.float(),
		'region': aoi,
		'crs': f'EPSG:{crs_epsg}',
		'scale': 1000,    
		'fileFormat': 'GeoTIFF',
		'fileNamePrefix': f'mali_test/{product_id}',
		'bucket':   f'mali_test',  
		'maxPixels':1e10,
		'description': 'test'
	}
	task=ee.batch.Export.image.toCloudStorage(**export_config)
	task.start()

In [19]:
img_list = era5_2mt.toList(era5_2mt.size())
nImages = img_list.size().getInfo()
for i in range(20):
  img = ee.Image(img_list.get(i))
  #img=ee.Image(era5_2mt.get(0)).clip(aoi)
  print(img.get('PRODUCT_ID').getInfo())

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [48]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [49]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))